In [1]:
import re 
from retrieval_importance.demo import load_wikifact_questions, BingRetriever, GPT35Generator, score_no_retrieval, \
    score_with_retrieval_augmentation, score_with_ragbooster

In [2]:
questions = load_wikifact_questions()
questions[0]

Question(text='Bill Belzer was born in', correct_answers=['Kansas City', 'Kansas City, MO', 'KCMO', 'KC', 'Kansas City, Missouri', 'K.C.', 'Kansas City', 'Kansas City'])

In [3]:
class MyLLMGenerator(GPT35Generator):
    
    FEW_SHOTS = "\nJerry Beck (born February 9, 1955, in New York City) is an American animation historian," +\
                " author, blogger, and video producer.Beck wrote or edited several books on classic" +\
                " American animation and classic characters.\nJerry Beck was born in New York\n\n" +\
                "Ettore Maria Fizzarotti (1916–1985) was an Italian film director and screenwriter." +\
                " Born in Naples, the son of the director Armando, he debuted as assistant director" +\
                " in the films of his father.\nEttore Maria Fizzarotti was born in Naples\n"    
    
    def _create_prompt(self, question, retrieved_text):        
        if retrieved_text is None:
            return f"{self.FEW_SHOTS}\n\n{question.text}"            
        else:
            return f"{self.FEW_SHOTS}\n\n{retrieved_text}\n\n{question.text}"             
        
    
    def _extract_answer(self, response):
        answer = response.get_response()          

        answer = re.sub(r'[0-9]+', '', answer)
        answer = answer.strip()   

        if ',' in answer:
            answer = answer.split(',')[0]
            
        if '.' in answer:
            answer = answer.split('.')[0]

        answer = answer.strip()  
        return answer     

In [4]:
class MyBingRetriever(BingRetriever):
    def create_query(self, prompt):
        return prompt

In [5]:
retriever = MyBingRetriever()
generator = MyLLMGenerator()

In [6]:
validation_questions = questions[:500]
test_questions = questions[500:1000]

In [7]:
score_no_retrieval(test_questions, generator)

  0%|          | 0/500 [00:00<?, ?it/s]

0.048

In [8]:
score_with_retrieval_augmentation(test_questions, retriever, generator, k=1)

  0%|          | 0/500 [00:00<?, ?it/s]

0.336

In [9]:
score_with_retrieval_augmentation(test_questions, retriever, generator, k=10)         

  0%|          | 0/500 [00:00<?, ?it/s]

0.49

In [10]:
score_with_ragbooster(validation_questions, test_questions, retriever, generator, k=10)             

Computing validation corpus...


  0%|          | 0/500 [00:00<?, ?it/s]

Learning importance weights for data sources...
Tuning threshold for corpus pruning...
Computing pruned predictions...


  0%|          | 0/500 [00:00<?, ?it/s]

0.516